In [ ]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [ ]:
!pip install keras-image-helper

In [16]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [17]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [18]:
from keras_image_helper import create_preprocessor

In [19]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [29]:
url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMSEhUSEhIVFhUXFhgZGBgYEhUXGBgXFxcWFxUVGBcYHSggGBolGxUXITEhJSkrLi4uGB8zODMsNygtLisBCgoKDg0OGhAQFy4dHR0rLS0tLS0tLS0rLS0rLS0tLS0tLS0tLS0rLS0rLS0tLS0rKystLS0rLS0tLS0rLSsrK//AABEIAPgAywMBIgACEQEDEQH/xAAcAAEAAgMBAQEAAAAAAAAAAAAABAcDBQYCAQj/xAA8EAACAQIDAwkFBgYDAQAAAAAAAQIDEQQSIQUxQQYiUWFxgZGhsQcTUsHRFCMkMkJyYoKSouHwM7Lxwv/EABgBAQEBAQEAAAAAAAAAAAAAAAACAQME/8QAHhEBAQACAwADAQAAAAAAAAAAAAERMQIDIRIyQSL/2gAMAwEAAhEDEQA/ALlABKgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAx4irkjKT/Sm/BXsYMgOWwnLSEtZ0pJXaTjJSvZtXs7aaG6we2aFX8lWN+hvLLwlZmTlKYTweYVE9zT7Gme7FD4D45Jb2vE+36wAPM6sY6uSS62l6kKvtvDQTcq9PTelNSf8ATG7ME8HIbQ5fUoaUqU6j6W8i+b8kTeR3KhY2NRSgoVKcrOKd04tXjJX70+gycpTDogAUAAAAAAAAAAAAAAAABznL/EzhhHGk7TqVKdNNb4qUudJdaim+46M53l3BSw3u3ulJX1s7JN6PhrYnlojhppU4qKVkkkl0JaJEd1LkDG7MlTs6NapHqlLPHwZApbRqwb9/T0X66bbT62t6PP8AHOqrLc1qUXqoxb+hqZUXUck4JRTSjlqSTuvzN5bf7fpJlLEqcc8Lyjbekt2q42e+5FwkZqMsy3zqO27mucsvlbxEzGtNtbY9GMJTyubim7Obtp17+PkfdnbNpVI05+6lHNp/yS6+vdp4Gx2pTVSm6clZSVtLacVw13DA08sYwi28mmZtvvtuL+dxtmHytgMqvTo0leyzSd2r9VtbdbNnUrJaRtZf73GCpmacIq909bpJaNcXfieMPDnTcndxk4pb+tSl12asu/sjOWvlfEpLV/8AvQibyI2g6O0aT/KqkZxknZOyTkrr69BrqmPhCSjBe8rPS/CPV1IybOSp4qNowSvG8owSbzc2Wq14srj5WVfiYPGHtljZ3VlZ9Vj2ehIAAAAAAAAAAAAAAAAcjy2rNzhTvzVHNbr1Tv5HXHD8rp3ry42SSXDcmc+36tjlcVNLTe3uRip0H38ddF2vp6iWqSTbveT3yt5R6CPjJq2Xcujj3nmUgVZr8kfy3u7K13w7tRmueK1ZRWiGFjKeqjZdJQxVaWaaXU36L5mWlRSWnH/bmX3f3i/bL1ifKGq7b28WBlg0tERpYdttrjvfdb5EmnRctx9+y24mCHDCqLvbwI20Kby3W9PT6k7ETUVq/wDwizrxknZPdx3GwXPyVxXvcLSl/CjbHGeyvFZsLKnfWnUa7nrHy9Dsz1S+IAAaAAAAAAAAAAAAAAV3yjxN687K7cmurTT5FhylZX6CpcbUcqkpN6tu3ZwOPbpseFXlJcEmuBAxc1FNtEmEsqS7P8mq5QTs1Hi7HGKR6tRy4WV0b/D08tPsRraVGNLKpLNVlqo/CviZtMZzaMr73F+L3CjV4N5oKo3+bN4afQ+7NTlTU312Xa9G/FmHZUr0IKzeVNPo0tH5My7Kr5tNEsrVv4k+3qNo2Ge14R0sl26mtnOTvqzLhZvPNvVv5aL0Mck1Ukkt6TXmn6GCPDC3d5EmatG0d/D6nuNF25z8DxVqxjpre12knJ5U0nJ23RV1eT0VzdjfezXGe4xPupPSsmtXduavKL7+cu8tY/P8MVlr01CX3iqRatwakmrvcu8v6nPMlLpSfirnfr0mvQAOjAAAAAAAAAAAAABH2lUy0akuiEn/AGsqRSvctPlAvw1b9jKsbSmopcNTz9u1cWJ3Tvfj66EfGQXv5Vp/8dCmu+bTlbuTRMqwblZdBp+W+LywVKO+b/8AX4nOe1rPycg5t4iprOo20t+WPBdWh55UYnLFSfB2W/e/8Jmw2dTVOlFLhFehpNs87DVJvW1rfuuvqbNiFttP7N7uL0qVqeX+ZRbvbgpZmSdj4jJifc2bThpJ2vdJZm7aK+mh7q0/eU6F9/6V/K2/RGfA4ZRxFJRWrp1W3a250or1ZX4xKU1BTnLcR3JzjCpu1kt70i7W17ka/amIdat7qL5kXrbi+LNvThzVHgTjDUPFYqvG0YUlU4ZnPJbrayvyPeCpVZNyqyWqccsE0rPfmk3d91jLR32fAy4fe3wW43ONCPTtGvR0SXvKeltLKa0L6KExatUpP+OP/ZF803ouxeh169Jr0ADqwAAAAAAAAAAAAARdqwzUai/gl6FP1p2qt9dvkXRWjeMl0prxRSe0Y/fQppc6ctXbdFN3OHbtXFslHVdP+s1UtjZ8R76vKLUVaEFuS6W3verZO2tKcZZacsumjsr9eu80i2XVb/5X25n46nKNbSFTfB74+ceDNHt6/wBjnFLXOvBat9hOo0MvNzXkm9b3fWnfgfdrTX2Sqtzlo+1u/pc2bEvkns54irBJaQgor901d+EVfvMG219mrVYJt5M0Lrfb3kmoRXxS5vYossT2Z7HVDCQnLWpNXl1N6tei7jm+WGEjSxNao1q2pptJ70npfjeTR05THGJjk9ibLlFOdRWlJ3fV1G0lTS4keHvKizN5V1b3/g91Ixgrt36EcqpjxCtzk92rM1GGlunUjU6Lm2mt9+7SxOqWpx1aWgGv2g/vKP74/wDZF64Z8yP7V6IoOaz1qL4OcbdfPRfuHVoxX8K9Dv16TXsAHVgAAAAAAAAAAAAA+lTVcJ+KqVH+nmR7t7LYK329HJiZ9DlK/jdepx7tN4tDt+dqnmKGqv1GPlCm5KST16PA9U3aNndO26xxU1OMxajXg01ld0+19Z9xWG95XhTe7Or+vyZptq4pPRON076au63bjoOS2CrV8TTtFtuWa90lpF6avoZcmmLj5ORtQj2uxyftJw16lKT/ACyjZ9eR3t5o7vDUVCKityVjR8t8Jnw+bjTkpd25+bXgduc/lMVxiauVbuxEOFJvV6tk3EU1e7V7bujt6xSdk6kuG48y2OvP3MdPzPe+g10IOpJOTuvUjV8Q6s78L2NthaagruyXXZI3Qi0ql8TTj8NWPcrQt6l709y7Efn/AGFU95iMy/VWVuxNfQ/QEFZLsO/XpNfQAdGAAAAAAAAAAAAAAV5yjgpYio9fzeiSfmiwyuds1o06lSU2lac+O95n5HHt03i0+1KsaVPPJ2S6d7dtElxd7HGS2riK75rtH5dpu9qShiZJyeZLRJTsl3Hyjh4xTSVjnMRTnaFOKnHmR1dt2ie7cWRyI0xdFfwVH5JfM4jZWDUpvPweZJPofHyOy5M1suNpPglJdzlBFT7Rn4tYxYqipwlB7pRa8UZQd0qdx7jCOaclGKtd38l1nM7R2vKtzKNOeVbnlau+mz3LtN9tKnF1ZOU01CUlC+5JNrNZcWuJhoShe1876IrTvPLPFsWx9m5FnqO8uCNRtzaHvqqpw3K6fzPm3uUTk3Touy3OS49SfR1mDYmCa50t7Kkx7WOh5KYNfaaMV8V34fVovQpnkJUzY2npo03/AHQ+hcp14aZQAHRgAAAAAAAAAAAAAFYcusHGdapmitJaXS4qLfmyzyueWLUsRU03OK7ebqcu3TeLgqmHcPyO1uD3Cljf0zWV+XibHFZb2VyFiMOpXV1foa+ZyUjYGNqqSeqavppa/O8jqNgyTxDtvUXbva/x4HEY6M6cs27S17X/AN3m55LYyUcTTv8AllaMu2VreDt5lSe5TV+x3LsPGIm4wlJK7UW0utJtGSx9O7FD1pUoa1Z9yTu/Ij4+hVrL3cGqdK3Ocd8urs7Te7doqOImklbPOy4aSkkjW1qc27pxtxTi/kzyraPB7DSlbNextbKKavrF6+C+p7tkabtF7t+hrMZi6cFJKTqVJcf0q/Qjfax0/s1ouePi/wBMYTfa1YuQqP2Xv8XFdNKp/wDJbh34aTQAFgAAAAAAAAAAAAAFfctaWXEt/FGL8nF+hYJxXtDw95UZW33T/lakr+Jz7J/LY4yrO2802P1bfTYz4iEsz5777W8iNVlwkjhFFOpdW366JIl4aajJO1nF69zuiNFq14pdxJ0bjPTVNP8A3tRovkIhbFxXvcPSqfFTi++yv5k1HpQp3bsr1Jvjnm/7mQqctCRtRfeTv8cv+zItPceVbU7VrJSd7t206F0GuwGFbd5LrN3jKUc6lLirJdLMUEry03R8blS+MdF7NZfjYfsqfIt8p32aS/F0v2yXki4jtw0ygALYAAAAAAAAAAAAABy/tBh9xTl0VLeKf0OoNBy5pZsJLqnB+eX5kc/rSKpx6tK5gcrmbadRJLtIuFvJ6LTpPOt9+zx32fiz7JNKajfWLsuN0tV3omKklvZ9nUyp2VuviMi1+RUbYDC9dGD8Vf5m7Rzns/xOfA0V8Cyd0dEdGj1TSFLbSf3k2/jl6shVK6jrJ26Fvb7Et5Ix8m5y0a50u3e+kiwpxTutX0t3Z5VosaUpy95PmrdGPFLr62Yq83nUIb5b30JbyXXTk7XskvMi4iChBuP5paX6Fx8TYxu/ZnL8clwUp+Fi6imPZfR/Gr9r+Rc534aTQAFgAAAAAAAAAAAAAGu5R4f3mFrRW/I2u2POXobENX0fEyih62Ccnq9F4skwhZWiesRRcZzTeqk14Nr5EWvXcU0t55VslatGGreprFXlVnZaL/dTzKm5a6vtJWBjlvfezdMWX7L5/h5R4Kcrf1SR2ZyXs2t9murcE/3K+f8AuudaemaSpzlHTcMTWjbT3k7dmZ28jVSm/hZ0vtBiljJW4xg+/Lb5HL1Ktt55rPVPinJ/pt3oh427e/RGeNbRye5bl0vckiJiFlVm9d7fySNg7D2TwzYqb+GHrf6FuFU+xql97Xb+CPq/qWsd+Ok0ABQAAAAAAAAAAAAAAAApflTNxxNaEb397NvqTk35mnk7LVnccteTNeVerXhFOm8r0kr3ypPm9qOap7BqPflXe2ef43KstTGvLoMimb6lyZqP/EWS6XJGUnrn8Eh8KZbL2WYrnVYL8raf82VX8ixDleSnJx4aV0rJ3bu7tu1kdUduMxEql5eVfxtVvhlS/oju8TlrOT9ew7nl3sKvPEzrQpt02oc5OO9RSel78DnKWxa3wf3R+pxsuVNdUir9SXnwMNSN3fj2m4ewaz/SvE+Lk3Xve0f6n9B8aOi9kVO0677F4a/Mssr3kBs2phqk87i88rrK27aWd7rsLCO3HSaAAoAAAAAAAAAAAAAAAAGjCsJBO+ReBmAHxQXQvA+2AAAABJX0Zg+x0/gXgZwBGls+m/0o8LZtP4UTAGMNLCwjqkZgA0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAH//Z'
X = preprocessor.from_url(url)

In [30]:
X

array([[[[0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         ...,
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ]],

        [[0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         ...,
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ]],

        [[0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         ...,
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ]],

        ...,

        [[0.92941177, 0.8901961 , 0.8666667 ],
         [0.92941177, 0.8901961 , 0.8666667 ]

In [31]:
def np_to_protobuf(numpy_array):
    return tf.make_tensor_proto(numpy_array, shape=numpy_array.shape)

In [32]:
np_to_protobuf(X)

dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 1
  }
  dim {
    size: 299
  }
  dim {
    size: 299
  }
  dim {
    size: 3
  }
}
tensor_content: "\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?\344\343c?\336\335]?\356\355m?

In [33]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [34]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [35]:
preds = pb_response.outputs['dense_7'].float_val

In [36]:
preds

[-2.941814661026001, -2.6534173488616943, -2.8301899433135986, -0.31378233432769775, 5.738794326782227, -2.3210690021514893, -2.996525764465332, 5.061158657073975, -2.1988651752471924, -2.914808988571167]

In [37]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [38]:
dict(zip(classes, preds))

{'dress': -2.941814661026001,
 'hat': -2.6534173488616943,
 'longsleeve': -2.8301899433135986,
 'outwear': -0.31378233432769775,
 'pants': 5.738794326782227,
 'shirt': -2.3210690021514893,
 'shoes': -2.996525764465332,
 'shorts': 5.061158657073975,
 'skirt': -2.1988651752471924,
 't-shirt': -2.914808988571167}